In [69]:
import cv2
import numpy as np
import math
import pandas as pd

In [2]:
def line_intersection(line1, line2):
    """
    Usage: let A = (x1, y1), B = (x2, y2), C = (x3, y3), D = (x4, y4)
            let l1 = (A, B), l2 = (C, D)
            call line_intersection(l1, l2)
            or directly call line_intersection((A, B), (C, D))
    Args: 
        line1 (2 points tuple): (A, B): A is first point coordinates (x11,y11), B is second point coordinates (x12,y12)
        line1 (2 points tuple): (C, D): C is first point coordinates (x21,y21), D is second point coordinates (x22,y22)
    Returns:
        is_intersecting (Boolean): True if intersection found, else False
        (x, y) (tuple of integers): coordinates of intersection point if found, else None
    """
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return False, (None, None)
    else:
        d = (det(*line1), det(*line2))
        x = det(d, xdiff) / div
        y = det(d, ydiff) / div
        return True, (x, y)

In [3]:
def detect_lines(img_in):
    img = img_in.copy()
#    cv2.imshow('img_in',img)
#    cv2.waitKey(0)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#    cv2.imshow('gray', gray)
#    cv2.waitKey(0)
    edges = cv2.Canny(gray, 50, 200, 5)
#    cv2.imshow('edges', edges)
#    cv2.waitKey(0)
    """
    lines_p = cv2.HoughLinesP(edges, rho = 1, theta = 1*np.pi/180, threshold = 35 ,minLineLength = 10, maxLineGap = 25)
    print("Hough lines P: ")
    if lines_p is not None:
        for line in lines_p:
            print(line)
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

#    cv2.imshow('houghlines P',img)
#    cv2.waitKey(0)
    """
    
#    img2 = img_in.copy()
    lines = cv2.HoughLines(edges, rho = 1, theta = 1*np.pi/180, threshold = 20)
    extract_lines = []
    print("Hough lines: ")
    if lines is not None:
        for line in lines: 
            for rho,theta in line:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                theta_deg = round(np.rad2deg(theta))
#                cv2.line(img2,(x1,y1),(x2,y2),(0,0,255),2)
                ext_line = ((x1,y1), (x2,y2), theta_deg)
                print(line, ext_line)
                extract_lines.append(ext_line)
    
#    cv2.imshow('houghlines',img2)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    return extract_lines

In [4]:
red_bounds = ([0, 0, 60], [50, 50, 255])          # red
orange_bounds = ([0, 100, 200], [50, 200, 255])   # orange
yellow_bounds = ([0, 150, 100], [50, 255, 255])   # yellow
white_bounds = ([250, 250, 250], [255, 255, 255]) # white

In [5]:
######### TODO define this function
def get_valid_sign_squares(lines):
    """
    input: list of lines in the format (see the caller in stop_sign_detection)
    return: list of squares in the format [[(x1,y1), side1], [....]]
    """
    return [[(0,0),0]]

In [13]:
def stop_sign_detection(img_in):
    """Finds the centroid coordinates of a stop sign in the provided
    image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the stop sign.
    """
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = red_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)
    """
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    #mask = cv2.dilate(mask,kernel,iterations = 1)
    cv2.imshow("mask", mask)
    cv2.waitKey(0)
    """
    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)
#   out_img = cv2.dilate(out_img,kernel,iterations = 3)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img,allowed_lines

In [14]:
def warning_sign_detection(img_in):
    """Finds the centroid coordinates of a warning sign in the
    provided image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the sign.
    """
    
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = yellow_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)

    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img

In [15]:
def construction_sign_detection(img_in):
    """Finds the centroid coordinates of a construction sign in the
    provided image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the sign.
    """
    
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = orange_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)

    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img

In [12]:
# ############# MILESTONE 2 #############

# #input_images = ['scene_yld_1', 'scene_stp_1', 'scene_constr_1', 'scene_wrng_1', 'scene_dne_1', 
# #                'scene_some_signs', 'scene_all_signs', 'noisy_scene_all_signs', 'scene_all_signs_noisy']
# input_images = ['scene_some_signs', 'scene_stp_1', 'scene_all_signs', 'scene_all_signs_noisy']

# for img_in in input_images:
#     image = cv2.imread("input_images/{}.png".format(img_in))
#     cv2.imshow("image", image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
#     ret_img = stop_sign_detection(image)
#     cv2.imshow("detected STOP sign", ret_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
#     ret_img = warning_sign_detection(image)
#     cv2.imshow("detected WARNING sign", ret_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

Hough lines: 
[[442.          1.5707964]] ((-1000, 441), (999, 442), 90.0)
[[457.          1.5707964]] ((-1000, 456), (999, 457), 90.0)
[[454.          1.5882496]] ((-1007, 436), (991, 471), 91.0)
[[439.          1.5882496]] ((-1007, 421), (992, 456), 91.0)
[[542.          0.7853982]] ((-323, 1090), (1090, -323), 45.0)
[[598.   0.]] ((598, 1000), (598, -1000), 0.0)
[[197.          1.5707964]] ((-1000, 196), (999, 197), 90.0)
[[423.          0.8552113]] ((-477, 975), (1032, -336), 49.0)
[[138.          1.5707964]] ((-1000, 137), (999, 138), 90.0)
[[-324.          2.338741]] ((-494, -927), (944, 461), 134.0)
[[-251.           2.3911011]] ((-498, -902), (865, 560), 137.0)
[[532.           0.82030475]] ((-368, 1071), (1094, -292), 47.0)
[[445.         1.553343]] ((-992, 462), (1007, 427), 89.0)
[[-341.           2.3736477]] ((-449, -956), (939, 482), 136.0)
[[499.   0.]] ((499, 1000), (499, -1000), 0.0)
[[428.           0.83775806]] ((-456, 987), (1029, -351), 48.0)
[[462.          1.53588

Hough lines: 
[[196.          1.5358897]] ((-992, 230), (1006, 160), 88.0)
[[-246.           3.1241393]] ((228, -1004), (263, 995), 179.0)
[[2.5400000e+02 3.4906585e-02]] ((218, 1008), (288, -990), 2.0)
[[185.          1.5882496]] ((-1003, 167), (996, 202), 91.0)
Hough lines: 
[[-262.           2.3561945]] ((-521, -892), (892, 521), 135.0)
[[598.   0.]] ((598, 1000), (598, -1000), 0.0)
[[297.          1.5707964]] ((-1000, 296), (999, 297), 90.0)
[[238.          1.5707964]] ((-1000, 237), (999, 238), 90.0)
[[-163.           2.3561945]] ((-591, -822), (822, 591), 135.0)
[[-183.           2.3911011]] ((-548, -856), (815, 606), 137.0)
[[6.0200000e+02 1.7453292e-02]] ((584, 1010), (619, -989), 1.0)
[[-252.          2.338741]] ((-544, -875), (894, 513), 134.0)
[[499.   0.]] ((499, 1000), (499, -1000), 0.0)
[[609.           0.80285144]] ((-296, 1132), (1142, -256), 46.0)
[[605.           0.82030475]] ((-318, 1124), (1143, -239), 47.0)
[[199.          1.5707964]] ((-1000, 198), (999, 199), 90.

Hough lines: 
[[297.          1.5707964]] ((-1000, 296), (999, 297), 90.0)
[[316.          1.5358897]] ((-988, 350), (1010, 280), 88.0)
[[287.          1.5882496]] ((-1004, 269), (994, 304), 91.0)
[[278.          1.6057029]] ((-1009, 242), (989, 312), 92.0)
[[307.         1.553343]] ((-994, 324), (1005, 289), 89.0)
[[325.          1.5184364]] ((-981, 376), (1015, 272), 87.0)
[[268.          1.6231562]] ((-1012, 215), (984, 319), 93.0)
[[335.          1.5009831]] ((-974, 403), (1020, 264), 86.0)
[[-487.          3.106686]] ((451, -1016), (521, 982), 178.0)
[[515.          0.7853982]] ((-342, 1071), (1071, -342), 45.0)
[[-483.           3.0892327]] ((430, -1023), (534, 973), 177.0)
Hough lines: 
[[299.          1.5707964]] ((-1000, 298), (999, 299), 90.0)
[[308.          1.5707964]] ((-1000, 307), (999, 308), 90.0)
[[-239.           2.6179938]] ((-293, -985), (706, 746), 150.0)
[[317.         1.553343]] ((-994, 334), (1005, 299), 89.0)
[[339.          1.5707964]] ((-1000, 338), (999, 339

Hough lines: 
[[-351.           2.3561945]] ((-458, -955), (955, 458), 135.0)
[[8.460000e+02 7.853982e-01]] ((-108, 1305), (1305, -108), 45.0)
[[7.860000e+02 7.679449e-01]] ((-129, 1265), (1260, -173), 44.0)
[[-337.          2.338741]] ((-485, -937), (953, 452), 134.0)
[[-365.           2.3736477]] ((-432, -972), (957, 465), 136.0)
[[775.           0.80285144]] ((-180, 1252), (1257, -137), 46.0)
[[-271.          2.338741]] ((-531, -889), (907, 499), 134.0)
[[8.4000000e+02 8.0285144e-01]] ((-135, 1298), (1302, -90), 46.0)
[[781.          0.7853982]] ((-154, 1259), (1259, -154), 45.0)
[[8.510000e+02 7.679449e-01]] ((-82, 1310), (1306, -128), 44.0)
[[-299.           2.3736477]] ((-479, -927), (909, 511), 136.0)
[[-286.           2.3561945]] ((-504, -909), (909, 504), 135.0)
[[769.           0.82030475]] ((-206, 1244), (1255, -119), 47.0)
[[-379.           2.3911011]] ((-404, -989), (959, 472), 137.0)
[[7.9100000e+02 7.5049156e-01]] ((-103, 1270), (1260, -191), 43.0)
[[299.          1.5707

[[87.        1.727876]] ((-1001, -70), (974, 242), 99.0)
[[37.         1.7976891]] ((-982, -188), (966, 261), 103.0)
[[2.       1.850049]] ((-961, -273), (960, 277), 106.0)
[[17.        1.850049]] ((-965, -259), (956, 291), 106.0)
[[5.        1.8675023]] ((-957, -287), (954, 297), 107.0)
[[-131.          2.042035]] ((-831, -570), (950, 337), 117.0)
[[-389.           2.3911011]] ((-397, -996), (966, 466), 137.0)
[[-453.           2.6529005]] ((-69, -1095), (869, 670), 152.0)
[[-515.           2.7925267]] ((141, -1115), (825, 763), 160.0)
[[-526.           2.8099802]] ((171, -1116), (822, 774), 161.0)
[[-615.           3.0892327]] ((561, -1030), (666, 966), 177.0)
[[-620.          3.106686]] ((584, -1021), (654, 977), 178.0)
[[-626.           3.1241393]] ((608, -1010), (643, 988), 179.0)
[[637.   0.]] ((637, 1000), (637, -1000), 0.0)
[[6.450000e+02 8.726646e-02]] ((555, 1052), (729, -939), 5.0)
[[6.5800000e+02 2.7925268e-01]] ((356, 1142), (908, -779), 16.0)
[[6.6700000e+02 2.9670596e-01

[[6.4900000e+02 5.7595867e-01]] ((0, 1192), (1088, -485), 33.0)
[[6.3000000e+02 6.1086524e-01]] ((-57, 1180), (1089, -457), 35.0)
[[6.3700000e+02 6.1086524e-01]] ((-51, 1184), (1095, -453), 35.0)
[[6.5800000e+02 6.2831855e-01]] ((-55, 1195), (1120, -422), 36.0)
[[608.          0.6632251]] ((-136, 1162), (1094, -413), 38.0)
[[651.          0.6632251]] ((-102, 1188), (1128, -387), 38.0)
[[577.          0.6981317]] ((-200, 1136), (1084, -395), 40.0)
[[586.          0.6981317]] ((-193, 1142), (1091, -389), 40.0)
[[600.          0.7679449]] ((-263, 1136), (1126, -302), 44.0)
[[552.          0.7853982]] ((-316, 1097), (1097, -316), 45.0)
[[554.          0.7853982]] ((-315, 1098), (1098, -315), 45.0)
[[617.          0.7853982]] ((-270, 1143), (1143, -270), 45.0)
[[609.           0.80285144]] ((-296, 1132), (1142, -256), 46.0)
[[544.          0.8552113]] ((-397, 1066), (1111, -245), 49.0)
[[624.          0.8552113]] ((-345, 1126), (1164, -185), 49.0)
[[520.           0.87266463]] ((-431, 1041)

[[-428.          2.565634]] ((-185, -1071), (903, 605), 147.0)
[[-457.           2.6005406]] ((-123, -1092), (906, 621), 149.0)
[[-477.          2.687807]] ((-9, -1107), (867, 689), 154.0)
[[-482.           2.7052603]] ((14, -1110), (859, 702), 155.0)
[[-461.           2.7052603]] ((-4, -1101), (840, 711), 155.0)
[[-469.           2.7227137]] ((21, -1104), (835, 722), 156.0)
[[-507.         2.75762]] ((95, -1117), (844, 737), 158.0)
[[-504.         2.75762]] ((92, -1115), (841, 738), 158.0)
[[-506.           2.7750735]] ((114, -1114), (830, 752), 159.0)
[[-615.           2.8797932]] ((335, -1125), (852, 806), 165.0)
[[-569.           2.8972466]] ((310, -1107), (794, 832), 166.0)
[[-584.           3.0019662]] ((439, -1071), (717, 908), 172.0)
[[-620.          3.054326]] ((530, -1050), (704, 942), 175.0)
[[-625.           3.0717795]] ((553, -1041), (693, 953), 176.0)
[[-622.           3.0717795]] ((550, -1040), (690, 954), 176.0)
[[-619.           3.0717795]] ((547, -1040), (687, 954), 1

[[6.280000e+02 6.981317e-02]] ((556, 1041), (696, -953), 4.0)
[[6.4700000e+02 1.5707964e-01]] ((482, 1088), (795, -886), 9.0)
[[6.6900000e+02 2.0943952e-01]] ((446, 1117), (862, -839), 12.0)
[[6.7800000e+02 2.0943952e-01]] ((455, 1119), (871, -837), 12.0)
[[6.540000e+02 2.268928e-01]] ((412, 1121), (862, -827), 13.0)
[[6.700000e+02 2.268928e-01]] ((427, 1125), (877, -823), 13.0)
[[6.560000e+02 2.443461e-01]] ((394, 1128), (878, -811), 14.0)
[[6.800000e+02 2.617994e-01]] ((398, 1141), (915, -789), 15.0)
[[6.8100000e+02 2.7925268e-01]] ((378, 1148), (930, -773), 16.0)
[[7.0100000e+02 3.4906584e-01]] ((316, 1179), (1000, -699), 20.0)
[[6.7800000e+02 3.6651915e-01]] ((274, 1176), (991, -690), 21.0)
[[6.5400000e+02 3.8397244e-01]] ((231, 1172), (980, -682), 22.0)
[[6.7600000e+02 4.0142572e-01]] ((231, 1184), (1012, -656), 23.0)
[[304.           0.41887903]] ((-129, 1037), (684, -789), 24.0)
[[6.6300000e+02 4.1887903e-01]] ((198, 1183), (1012, -643), 24.0)
[[306.           0.43633232]] ((-14

Hough lines: 
[[8.460000e+02 7.853982e-01]] ((-108, 1305), (1305, -108), 45.0)
[[-351.           2.3561945]] ((-458, -955), (955, 458), 135.0)
[[7.860000e+02 7.679449e-01]] ((-129, 1265), (1260, -173), 44.0)
[[-337.          2.338741]] ((-485, -937), (953, 452), 134.0)
[[-365.           2.3736477]] ((-432, -972), (957, 465), 136.0)
[[-271.          2.338741]] ((-531, -889), (907, 499), 134.0)
[[775.           0.80285144]] ((-180, 1252), (1257, -137), 46.0)
[[8.4000000e+02 8.0285144e-01]] ((-135, 1298), (1302, -90), 46.0)
[[8.510000e+02 7.679449e-01]] ((-82, 1310), (1306, -128), 44.0)
[[-299.           2.3736477]] ((-479, -927), (909, 511), 136.0)
[[781.          0.7853982]] ((-154, 1259), (1259, -154), 45.0)
[[-286.           2.3561945]] ((-504, -909), (909, 504), 135.0)
[[769.           0.82030475]] ((-206, 1244), (1255, -119), 47.0)
[[-257.           2.3212879]] ((-556, -869), (906, 494), 133.0)
[[-379.           2.3911011]] ((-404, -989), (959, 472), 137.0)
[[7.9100000e+02 7.5049156

In [109]:
input_images = ['scene_some_signs', 'scene_stp_1', 'scene_all_signs', 'scene_all_signs_noisy']
input_images=["scene_some_signs"]
for img_in in input_images:
    image = cv2.imread("input_images/{}.png".format(img_in))
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ret_img,lines = stop_sign_detection(image)
    cv2.imshow("detected STOP sign", ret_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Hough lines: 
[[442.          1.5707964]] ((-1000, 441), (999, 442), 90.0)
[[457.          1.5707964]] ((-1000, 456), (999, 457), 90.0)
[[454.          1.5882496]] ((-1007, 436), (991, 471), 91.0)
[[439.          1.5882496]] ((-1007, 421), (992, 456), 91.0)
[[542.          0.7853982]] ((-323, 1090), (1090, -323), 45.0)
[[598.   0.]] ((598, 1000), (598, -1000), 0.0)
[[197.          1.5707964]] ((-1000, 196), (999, 197), 90.0)
[[423.          0.8552113]] ((-477, 975), (1032, -336), 49.0)
[[138.          1.5707964]] ((-1000, 137), (999, 138), 90.0)
[[-324.          2.338741]] ((-494, -927), (944, 461), 134.0)
[[-251.           2.3911011]] ((-498, -902), (865, 560), 137.0)
[[532.           0.82030475]] ((-368, 1071), (1094, -292), 47.0)
[[445.         1.553343]] ((-992, 462), (1007, 427), 89.0)
[[-341.           2.3736477]] ((-449, -956), (939, 482), 136.0)
[[499.   0.]] ((499, 1000), (499, -1000), 0.0)
[[428.           0.83775806]] ((-456, 987), (1029, -351), 48.0)
[[462.          1.53588

In [112]:
square_lines=[]
distances=[]
for i in range(len(lines)):
    for j in range(i+1):
        result=line_intersection(lines[i][:2],lines[j][:2])
        if result[0]:
            square_lines.append(result[1])


if len(square_lines)>=4:
    for i in range(len(square_lines)):
        temp=[]
        for j in range(len(square_lines)):
            x1,y1=square_lines[i]
            x2,y2=square_lines[j]
            dist = round(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ),3)
            temp.append(dist)
        distances.append(temp)
            
d=pd.DataFrame(distances)
display(d)


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000,98.995,97.581,139.004,36.770,104.278,156.978,184.835,504.874,514.488,506.211,528.715
1,98.995,0.000,139.004,97.581,62.225,115.732,57.983,113.508,514.488,504.874,508.694,508.193
2,97.581,139.004,0.000,98.995,104.278,36.770,184.835,156.978,407.294,419.152,408.950,436.497
3,139.004,97.581,98.995,0.000,115.732,62.225,113.508,57.983,419.152,407.294,412.019,411.400
4,36.770,62.225,104.278,115.732,0.000,97.581,120.208,154.829,506.211,508.694,504.874,518.987
5,104.278,115.732,36.770,62.225,97.581,0.000,154.829,120.208,408.950,412.019,407.294,424.662
6,156.978,57.983,184.835,113.508,120.208,154.829,0.000,97.581,528.715,508.193,518.987,504.874
7,184.835,113.508,156.978,57.983,154.829,120.208,97.581,0.000,436.497,411.400,424.662,407.294
8,504.874,514.488,407.294,419.152,506.211,408.950,528.715,436.497,0.000,98.995,36.770,156.978
9,514.488,504.874,419.152,407.294,508.694,412.019,508.193,411.400,98.995,0.000,62.225,57.983


In [111]:
# for i in square_lines:
#     print(i)
#     final=cv2.circle(ret_img, (int(i[0]),int(i[1])), radius=5, color=(255, 255, 255), thickness=-1)
#     cv2.imshow("draw center", final)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

(618.0, 149.0)
(548.0, 79.0)
(549.0, 218.0)
(479.0, 148.0)
(592.0, 123.0)
(523.0, 192.0)
(507.0, 38.0)
(438.0, 107.0)
(261.0, 506.0)
(191.0, 436.0)
(235.0, 480.0)
(150.0, 395.0)


In [108]:
d=pd.DataFrame(distances)
d

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000,98.995,97.581,139.004,36.770,104.278,156.978,184.835,504.874,514.488,506.211,528.715
1,98.995,0.000,139.004,97.581,62.225,115.732,57.983,113.508,514.488,504.874,508.694,508.193
2,97.581,139.004,0.000,98.995,104.278,36.770,184.835,156.978,407.294,419.152,408.950,436.497
3,139.004,97.581,98.995,0.000,115.732,62.225,113.508,57.983,419.152,407.294,412.019,411.400
4,36.770,62.225,104.278,115.732,0.000,97.581,120.208,154.829,506.211,508.694,504.874,518.987
5,104.278,115.732,36.770,62.225,97.581,0.000,154.829,120.208,408.950,412.019,407.294,424.662
6,156.978,57.983,184.835,113.508,120.208,154.829,0.000,97.581,528.715,508.193,518.987,504.874
7,184.835,113.508,156.978,57.983,154.829,120.208,97.581,0.000,436.497,411.400,424.662,407.294
8,504.874,514.488,407.294,419.152,506.211,408.950,528.715,436.497,0.000,98.995,36.770,156.978
9,514.488,504.874,419.152,407.294,508.694,412.019,508.193,411.400,98.995,0.000,62.225,57.983
